In [537]:
import pandas as pd

In [538]:
events = pd.read_csv('events.csv')
events_df = events.copy()

In [539]:
purchase = pd.read_csv('purchase.csv')
purchase_df = purchase.copy()

In [540]:
mask1 = (events_df['start_time']>='2018-01-01') & (events_df['start_time']<'2019-01-01') & (events_df['event_type']=='registration')
registered2018 = events_df[mask1]['user_id'].to_list()
events_df = events_df[events_df['user_id'].isin(registered2018)]

In [541]:
events_df['start_time'] = pd.to_datetime(events_df['start_time'], format='%Y-%m-%dT%H:%M:%S')

In [542]:
purchase_df = purchase_df[purchase_df['user_id'].isin(registered2018)]
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'], format='%Y-%m-%dT%H:%M:%S')

In [543]:
registered_users_count = events_df[events_df["event_type"] == "registration"][
    "user_id"
].nunique()
tutorial_start_users_count = events_df[events_df["event_type"] == "tutorial_start"][
    "user_id"
].nunique()
percent_tutorial_start_users = tutorial_start_users_count / registered_users_count
print(
    "Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_tutorial_start_users
    )
)

Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): 59.51%


In [544]:
tutorial_finish_users_count = events_df[events_df["event_type"] == "tutorial_finish"][
    "user_id"
].nunique()
tutorial_completion_rate = tutorial_finish_users_count / tutorial_start_users_count
print(
    "Процент пользователей, завершивших обучение: {:.2%}".format(
        tutorial_completion_rate
    )
)

Процент пользователей, завершивших обучение: 86.44%


In [545]:
level_choice_users_count = events_df[events_df["event_type"] == "level_choice"][
    "user_id"
].nunique()
percent_level_choice_users = level_choice_users_count / registered_users_count
print(
    "Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_level_choice_users
    )
)

Процент пользователей, выбравших уровень сложности тренировок (от общего числа зарегистрировавшихся): 41.86%


In [546]:
training_choice_users_count = events_df[events_df["event_type"] == "pack_choice"][
    "user_id"
].nunique()
percent_training_choice_users = training_choice_users_count / level_choice_users_count
print(
    "Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        percent_training_choice_users
    )
)

Процент пользователей, выбравших набор бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): 68.77%


In [547]:
paying_users_count = purchase_df["user_id"].nunique()
percent_of_paying_users = paying_users_count / training_choice_users_count
print(
    "Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): {:.2%}".format(
        percent_of_paying_users
    )
)

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали тренировки): 27.89%


In [548]:
purchase_rate = paying_users_count / registered_users_count
print(
    "Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей): {:.2%}".format(
        purchase_rate
    )
)

Процент пользователей, которые оплатили вопросы(от числа зарегистрировавшихся пользователей): 8.03%


In [549]:
purchase_df['event_type'] = 'purchase'
events_df = events_df.rename(columns={"id": "event_id"})
purchase_df = purchase_df.rename(columns={"id": "purchase_id"})
purchase_df = purchase_df.rename(columns={"event_datetime": "start_time"})
total_events_df = pd.concat([events_df,purchase_df],sort=False)


In [550]:
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

In [551]:
user_path_df = (
    total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index()
)
user_path_df.head(10)

,user_id,event_type
0,27832,[registration]
1,27833,"[registration, tutorial_start, tutorial_finish]"
2,27834,"[registration, tutorial_start, tutorial_finish]"
3,27835,"[registration, tutorial_start, tutorial_finish..."
4,27836,"[registration, tutorial_start, tutorial_start,..."
5,27837,[registration]
6,27838,[registration]
7,27839,"[registration, tutorial_start, tutorial_finish..."
8,27840,"[registration, tutorial_start, level_choice]"
9,27841,"[registration, tutorial_start, tutorial_finish]"


In [552]:
user_path_df["event_path"] = user_path_df["event_type"].apply(lambda x: " > ".join(x))
user_path_df["event_path"].head()

0                                         registration
1      registration > tutorial_start > tutorial_finish
2      registration > tutorial_start > tutorial_finish
3    registration > tutorial_start > tutorial_finis...
4    registration > tutorial_start > tutorial_start...
Name: event_path, dtype: object

In [553]:
user_paths = (
    user_path_df.groupby(["event_path"])["user_id"]
    .nunique()
    .sort_values(ascending=False)
)
user_paths.head(10)

event_path
registration                                                                                                       7970
registration > tutorial_start > tutorial_finish > level_choice > pack_choice                                       2796
registration > tutorial_start > tutorial_finish                                                                    1956
registration > tutorial_start > tutorial_finish > level_choice                                                     1713
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                            1083
registration > tutorial_start                                                                                       842
registration > tutorial_start > level_choice > pack_choice                                                          346
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish                                  323
registration > tutorial_start

In [554]:
user_paths[user_paths.index.str.contains('purchase')].head(10)

event_path
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                                             1083
registration > tutorial_start > level_choice > pack_choice > purchase                                                                                                                                124
registration > tutorial_start > tutorial_finish > level_choice > pack_choice > tutorial_start > tutorial_finish > purchase                                                                           101
registration > tutorial_start > tutorial_finish > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                            52
registration > tutorial_start > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase                                                                                 

In [555]:
registration_df = total_events_df[total_events_df['event_type'] == 'registration']
registration_df['user_id'].value_counts().mean()

1.0

In [556]:
registration_df = registration_df[["user_id", "start_time"]].rename(columns={"start_time":"registration_time"})

In [557]:
tutorial_start_df = total_events_df[total_events_df['event_type'] == 'tutorial_start']
tutorial_start_df['user_id'].value_counts().mean()

1.522179119581717

In [558]:
tutorial_start_df_wo_duplicates = tutorial_start_df.sort_values("start_time").drop_duplicates("user_id")
tutorial_start_df_wo_duplicates['user_id'].value_counts().mean()

1.0

In [559]:
tutorial_start_df_wo_duplicates = tutorial_start_df_wo_duplicates[["user_id", "tutorial_id", "start_time"]].rename(columns={"start_time":"tutorial_start_time"})

In [560]:
merged_df = registration_df.merge(tutorial_start_df_wo_duplicates, on="user_id", how="inner")

In [561]:
merged_df["timedelta"] = (merged_df["tutorial_start_time"]-merged_df["registration_time"])
merged_df.head()

,user_id,registration_time,tutorial_id,tutorial_start_time,timedelta
0,27833,2018-01-01 04:07:25,31508.0,2018-01-01 17:47:40,0 days 13:40:15
1,27834,2018-01-01 08:35:10,31510.0,2018-01-01 19:46:11,0 days 11:11:01
2,27835,2018-01-01 11:54:47,31506.0,2018-01-01 15:00:51,0 days 03:06:04
3,27836,2018-01-01 13:28:07,31505.0,2018-01-01 14:54:40,0 days 01:26:33
4,27839,2018-01-01 18:24:01,31509.0,2018-01-01 19:11:36,0 days 00:47:35


In [562]:
merged_df['timedelta'].describe()

count                        11858
mean     0 days 04:38:24.019817844
std      0 days 04:15:09.650705034
min                0 days 00:00:34
25%         0 days 01:21:37.500000
50%         0 days 03:22:08.500000
75%         0 days 06:47:42.750000
max                1 days 16:03:46
Name: timedelta, dtype: object

In [563]:
tutorial_finish_df = total_events_df[total_events_df['event_type'] == "tutorial_finish"]

In [564]:
first_tutorial_ids = tutorial_start_df_wo_duplicates['tutorial_id'].unique()

In [565]:
tutorial_finish_df = tutorial_finish_df[tutorial_finish_df['tutorial_id'].isin(first_tutorial_ids)]
tutorial_finish_df['user_id'].value_counts().mean()

1.0

In [566]:
tutorial_finish_df=tutorial_finish_df[["user_id", "start_time"]].rename(columns={"start_time":"tutorial_finish_time"})
merged_df_2=tutorial_start_df_wo_duplicates.merge(tutorial_finish_df, on="user_id", how="inner")
merged_df_2['timedelta']=(merged_df_2['tutorial_finish_time']-merged_df_2['tutorial_start_time'])
merged_df_2.head()

,user_id,tutorial_id,tutorial_start_time,tutorial_finish_time,timedelta
0,27835,31506.0,2018-01-01 15:00:51,2018-01-01 15:06:15,0 days 00:05:24
1,27833,31508.0,2018-01-01 17:47:40,2018-01-01 17:50:08,0 days 00:02:28
2,27839,31509.0,2018-01-01 19:11:36,2018-01-01 19:16:32,0 days 00:04:56
3,27834,31510.0,2018-01-01 19:46:11,2018-01-01 19:48:01,0 days 00:01:50
4,27845,31512.0,2018-01-02 03:03:44,2018-01-02 03:06:48,0 days 00:03:04


In [567]:
print(merged_df_2['timedelta'].mean())
print(merged_df_2['timedelta'].describe())

0 days 00:03:53.174160732
count                         9830
mean     0 days 00:03:53.174160732
std      0 days 00:01:44.803217992
min                0 days 00:00:16
25%                0 days 00:02:28
50%                0 days 00:03:42
75%                0 days 00:05:08
max                0 days 00:10:06
Name: timedelta, dtype: object


In [568]:
level_choice_df=total_events_df[total_events_df['event_type']=="level_choice"]
level_choice_df['user_id'].value_counts().mean()

1.0

In [569]:
level_choice_df=level_choice_df[["user_id", "start_time"]].rename(columns={"start_time":"level_choice_time"})
merged_df_3=registration_df.merge(level_choice_df, on="user_id", how="inner")
merged_df_3['timedelta']=(merged_df_3['level_choice_time']-merged_df_3['registration_time'])
merged_df_3.head()

,user_id,registration_time,level_choice_time,timedelta
0,27835,2018-01-01 11:54:47,2018-01-01 20:37:22,0 days 08:42:35
1,27839,2018-01-01 18:24:01,2018-01-01 22:37:50,0 days 04:13:49
2,27840,2018-01-01 20:53:21,2018-01-02 05:18:42,0 days 08:25:21
3,27842,2018-01-01 23:18:46,2018-01-02 08:46:03,0 days 09:27:17
4,27843,2018-01-02 00:02:28,2018-01-02 14:09:58,0 days 14:07:30


In [570]:
print(merged_df_3['timedelta'].mean())
print(merged_df_3['timedelta'].describe())

0 days 07:10:19.169863342
count                         8342
mean     0 days 07:10:19.169863342
std      0 days 04:33:51.164488800
min                0 days 00:08:15
25%         0 days 03:53:16.500000
50%                0 days 06:03:28
75%         0 days 09:34:58.500000
max                1 days 18:48:25
Name: timedelta, dtype: object


In [571]:
total_events_df['event_type'].value_counts()

registration       19926
tutorial_start     18050
tutorial_finish    14904
level_choice        8342
pack_choice         5737
purchase            1600
Name: event_type, dtype: int64

Practical part 38.7

In [572]:
pack_choice_df=total_events_df[total_events_df['event_type']=="pack_choice"]
pack_choice_df['user_id'].value_counts().mean()

1.0

In [573]:
pack_choice_df=pack_choice_df[["user_id", "start_time"]].rename(columns={"start_time":"pack_choice_time"})
merged_df_4=level_choice_df.merge(pack_choice_df, on="user_id", how="inner")
merged_df_4['timedelta']=(merged_df_4['pack_choice_time']-merged_df_4['level_choice_time'])
merged_df_4.head()

,user_id,level_choice_time,pack_choice_time,timedelta
0,27835,2018-01-01 20:37:22,2018-01-01 20:38:43,0 days 00:01:21
1,27839,2018-01-01 22:37:50,2018-01-01 22:42:54,0 days 00:05:04
2,27845,2018-01-02 06:19:18,2018-01-02 06:25:12,0 days 00:05:54
3,27849,2018-01-02 11:53:11,2018-01-02 11:59:26,0 days 00:06:15
4,27843,2018-01-02 14:09:58,2018-01-02 14:14:51,0 days 00:04:53


In [574]:
print(merged_df_4['timedelta'].mean())
print(merged_df_4['timedelta'].describe())

0 days 00:05:17.128464354
count                         5737
mean     0 days 00:05:17.128464354
std      0 days 00:02:43.923107804
min                0 days 00:00:19
25%                0 days 00:03:05
50%                0 days 00:04:57
75%                0 days 00:07:08
max                0 days 00:15:48
Name: timedelta, dtype: object


In [575]:
first_purchase_df=total_events_df[total_events_df['event_type']=="purchase"]
first_purchase_df['user_id'].value_counts().mean()

1.0

In [576]:
first_purchase_df=first_purchase_df[["user_id", "start_time"]].rename(columns={"start_time":"first_purchase_time"})
merged_df_5=pack_choice_df.merge(first_purchase_df, on="user_id", how="inner")
merged_df_5['timedelta']=(merged_df_5['first_purchase_time']-merged_df_5['pack_choice_time'])
merged_df_5.head()

,user_id,pack_choice_time,first_purchase_time,timedelta
0,27845,2018-01-02 06:25:12,2018-01-03 18:53:43,1 days 12:28:31
1,27865,2018-01-04 06:03:20,2018-01-04 14:46:10,0 days 08:42:50
2,27884,2018-01-04 16:22:03,2018-01-08 19:37:34,4 days 03:15:31
3,27910,2018-01-05 12:05:28,2018-01-07 12:11:34,2 days 00:06:06
4,27911,2018-01-05 17:40:37,2018-01-07 08:19:12,1 days 14:38:35


In [577]:
print(merged_df_5['timedelta'].mean())
print(merged_df_5['timedelta'].describe())

3 days 17:46:53.403125
count                         1600
mean        3 days 17:46:53.403125
std      2 days 04:37:20.225124289
min                0 days 00:44:50
25%         1 days 21:24:13.250000
50%         3 days 12:51:25.500000
75%         5 days 09:42:13.750000
max               10 days 18:33:59
Name: timedelta, dtype: object


In [578]:
users_with_finished_tutorial = total_events_df[total_events_df['event_type']=="tutorial_finish"]['user_id'].unique()
print(len(users_with_finished_tutorial))

10250


In [579]:
users_with_started_tutorial = total_events_df[total_events_df['event_type']=="tutorial_start"]['user_id'].unique()
set_users_with_started_tutorial = set(users_with_started_tutorial)
set_users_not_finished_but_started_tutorial = (set_users_with_started_tutorial.difference(set(users_with_finished_tutorial)))

print(len(set_users_with_started_tutorial))
print(len(set_users_not_finished_but_started_tutorial))
print(len(set_users_with_started_tutorial)-len(set(users_with_finished_tutorial)) == len(set_users_not_finished_but_started_tutorial))

11858
1608
True


In [580]:
all_users = total_events_df['user_id'].unique()
set_all_users = set(all_users)
set_users_not_started_tutorial = set_all_users.difference(set_users_with_started_tutorial)

print(len(set_users_not_started_tutorial))
print(len(set_all_users)-len(set_users_with_started_tutorial) == len(set_users_not_started_tutorial))

8068
True


In [581]:
len(set_users_not_finished_but_started_tutorial)+len(set_users_not_started_tutorial)+len(users_with_finished_tutorial) == len(set_all_users)

True

In [582]:
purchase_df_1 = purchase_df[purchase_df['user_id'].isin(users_with_finished_tutorial)]
purchase_df_1['user_id'].nunique()

1447

In [583]:
percent_of_purchase_1 = purchase_df_1['user_id'].nunique()/len(users_with_finished_tutorial)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение): {:.2%}".format(
        percent_of_purchase_1
    )
)

Процент пользователей, которые оплатили тренировки (от числа пользователей, завершивших обучение): 14.12%


In [584]:
purchase_df_1['amount'].mean()

110.98825155494126

In [585]:
purchase_df_2 = purchase_df[
    purchase_df["user_id"].isin(set_users_not_finished_but_started_tutorial)
]
print(purchase_df_2["user_id"].nunique())
percent_of_purchase_2 = purchase_df_2["user_id"].nunique() / len(
    set_users_not_finished_but_started_tutorial
)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших): {:.2%}".format(
        percent_of_purchase_2
    )
)

131
Процент пользователей, которые оплатили тренировки (от числа пользователей, начавших обучение, но не завершивших): 8.15%


In [586]:
purchase_df_2['amount'].mean()

104.9618320610687

In [587]:
purchase_df_3 = purchase_df[purchase_df['user_id'].isin(set_users_not_started_tutorial)]
print(purchase_df_3['user_id'].nunique())

percent_of_purchase_3 = purchase_df_3['user_id'].nunique()/len(set_users_not_started_tutorial)
print(
    "Процент пользователей, которые оплатили тренировки (от числа пользователей, не проходивших обучение): {:.2%}".format(
        percent_of_purchase_3
    )
)

22
Процент пользователей, которые оплатили тренировки (от числа пользователей, не проходивших обучение): 0.27%


In [588]:
purchase_df_3['amount'].mean()

128.4090909090909